In [47]:
import h5py
import pandas as pd
import numpy as np
import os

from tqdm import tqdm

In [48]:
def convert_to_hdf5(input_dir, output_file):
    file_names = [f for f in os.listdir(input_dir) if f.endswith('.parquet')]
    size = len(file_names) * 1000
    with h5py.File(output_file, 'w') as h5f:
        images_dset = h5f.create_dataset('images', (size, 1, 150, 150), dtype='float32')
        diffs_dset = h5f.create_dataset('diffs', (size,), dtype='int')
        
        idx = 0
        for file_name in tqdm(file_names):
            X = pd.read_parquet(os.path.join(input_dir, file_name)).to_numpy()
            diffs = X[1:] - X[:-1]
            diff_index = np.argmax(diffs, axis=1)
            diffs_dset[idx:idx + len(diffs)] = diff_index

            images = X.reshape(-1, 1, 150, 150)
            images_dset[idx:idx + len(diffs)] = images[:-1]
            idx = idx + len(diffs)

input_dir = '../data'
output_file = '../data/images_diffs.hdf5'
convert_to_hdf5(input_dir, output_file)

100%|██████████| 1000/1000 [26:37<00:00,  1.60s/it]
